In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0, '../scripts')
from simulation import *

In [59]:
N = 80000
p = 0.005
K = N*p
P = 16
tau = 0.01
dt = 1e-3
A = 20
theta = 0.07
sigma = 0.05
x_f = 1.5
q_f = 0.8

In [60]:
patterns = np.random.RandomState(seed=1).normal(0,1,size=(P,N))

In [35]:
# Structural connectivity
k = np.random.RandomState(seed=2).binomial(N, p, size=N)
ji = build_ji(k, N)

100%|██████████| 80000/80000 [01:51<00:00, 716.87it/s]


In [36]:
# Store pattern associations using the threshold plasticity rule
plasticity = ThresholdPlasticityRule(x_f, q_f)
indptr, indices, data_p0 = store_associations(
    patterns, plasticity.f, plasticity.g, ji, K, p=0)
_, _, data_p1 = store_associations(
    patterns, plasticity.f, plasticity.g, ji, K, p=1)

100%|██████████| 80000/80000 [00:07<00:00, 11129.41it/s]


In [61]:
# Adjust subpopulation weights
data_p1_copy = np.asarray(data_p1).copy()
data_p0_copy = np.asarray(data_p0).copy()
reweight(indptr, indices, data_p1_copy, # Asymmetric
         A, N, w_11=1, w_12=1, w_21=0, w_22=0)
reweight(indptr, indices, data_p0_copy, # Symmetric
         A, N, w_11=0, w_12=0, w_21=1, w_22=1)
data = data_p1_copy + data_p0_copy
W = cp.sparse.csr_matrix(
    (cp.asarray(data), cp.asarray(indices), cp.asarray(indptr)),
    shape=(N,N),
    dtype=cp.float32)

80000it [00:00, 140170.43it/s]
80000it [00:00, 137146.97it/s]


In [62]:
def run_simulation(
    I_ext_1,
    I_ext_2,
    T_transient = 0.01,
    T_preparatory = 0.2):
    
    
    N_1 = N_2 = int(N/2)
    
    # Sustained inputs
    def xi_sustained(t):
        if t < 0.2:
            xi_t_1 = cp.full(N_1, fill_value=-0.8) # Asymmetric input
            if t > 0.01:
                xi_t_2 = cp.full(N_2, fill_value=0) # Symmetric input
            else:
                xi_t_2 = cp.full(N_2, fill_value=0) # Symmetric input
        else:
            xi_t_1 = cp.full(N_1, fill_value=I_ext_1)
            xi_t_2 = cp.full(N_2, fill_value=I_ext_2)
        return cp.r_[xi_t_1 , xi_t_2]
    
    # Transient inputs
    def xi_transient(t):
        if t < 0.01:
            xi_t_1 = cp.full(N_1, fill_value=0)
            xi_t_2 = patterns[0,int(N/2):]
            return cp.r_[xi_t_1 , xi_t_2]
        else:
            return 0
        
    def xi_t(t):
        return xi_sustained(t) + xi_transient(t)
    
    r = simulate(
        T=0.8,
        dt=dt,
        tau=tau,
        N=N,
        I_ext_1=None,
        I_ext_2=None,
        I_ext=xi_t,
        theta=theta,
        sigma=sigma,
        patterns=patterns,
        r0=phi(cp.zeros(N), theta, sigma),
        W=W)
    
    _, q1, q2 = correlations(r, patterns, individual=True)
    
    return {
        'r': r.astype(np.float16),
        
        'q1': q1,
        'q2': q2,
        
        'params': {
            "N": N,
            "p": p,
            "K": K,
            "P": P,
            "A": A,
            "tau": tau,
            "theta": theta,
            "sigma": sigma,
            "x_f": plasticity.x_f,
            "q_f": plasticity.q_f,
            "x_g": plasticity.x_g,
            "q_g": plasticity.q_g,
            'I_ext_1': I_ext_1,
            'I_ext_2': I_ext_2,
        }
    }

In [63]:
directory = "4-state-transitions/data"

#### Short interval

In [64]:
data = run_simulation(I_ext_1=-.04, I_ext_2=-.2)
filepath = directory + "short_interval.npy"
np.save(open(filepath, 'wb'), data)

100%|██████████| 799/799 [00:01<00:00, 460.94it/s]


#### Long interval

In [65]:
data = run_simulation(I_ext_1=-.30, I_ext_2=-.05)
filepath = directory + "long_interval.npy"
np.save(open(filepath, 'wb'), data)

100%|██████████| 799/799 [00:01<00:00, 457.19it/s]
